In [1]:
import numpy as np
import pandas as pd
import torch
import json
import os
import pickle
from collections import OrderedDict
import psutil
import gc

In [2]:
def print_txt(msg: str):
    print(msg, end = '\r')
    text_file = open("Outputs/gaia_preprocessing.txt", "w")
    text_file.write(msg)
    text_file.close()

In [3]:
##directorys
input_path = 'data/scraped_matches/inputs/'
gaia_path = 'data/scraped_matches/gaia_data/'

In [4]:
#load data
with open('data/scraped_matches/parsed_matches/arabia_matches.pkl', 'rb') as f:
    org_matches = pickle.load(f)
matches = org_matches

#load masterdata
with open('masterdata/gaia_masterdata.pkl', 'rb') as f:
    gaia_masterdata = pickle.load(f)
with open('masterdata/gaia_masterdata_reversed.pkl', 'rb') as f:
    gaia_data_rev = pickle.load(f)
    
gaia_types = list(gaia_masterdata.keys())

In [5]:
gaia_types

['tree',
 'animal_gatherable',
 'animal_hostile',
 'animal_herdable',
 'aquatic_animal',
 'gold_mine',
 'stone_mine',
 'relic',
 'food',
 'p1_tc',
 'p2_tc']

In [6]:
match = matches['50453403']
match_id = '50453403'

In [7]:
gaia_matchids = []
gaia_labels = {}

for index,match_id in enumerate(matches):
    match = matches[match_id]


    with open(match['gaia_fn'], 'rb') as f:
        gaia_data = pickle.load(f)


    #init empty arrays of shape map size
    arrays = {}
    for type in gaia_types:
        arrays[type] = np.zeros(shape=(120,120))

    #read tc position
    arrays['p1_tc'][int(match['p1_ypos'])][int(match['p1_xpos'])] = 1
    arrays['p2_tc'][int(match['p2_ypos'])][int(match['p2_xpos'])] = 1

    for id in gaia_data:
        obj = gaia_data[id]
        try:
            #fill values into arrays
            type = gaia_data_rev[obj['name']]
            arrays[type][int(obj['y_pos'])][int(obj['x_pos'])] = 1
        except:
            pass

    array_list = []
    for i in arrays:
        array_list.append(arrays[i])

    md_array =  np.stack(array_list, axis=0)
    tensor = torch.tensor(md_array)
    
    #persistate tensor
    output = open(f'data/feature_maps/gaia/{match_id}.pkl' , 'wb')
    pickle.dump(tensor, output)
    output.close()
    
    #safe label and match_id
    gaia_labels[match_id] = match['winner']
    gaia_matchids.append(match_id)
    
    if index % 1000 == 0:
        print_txt(f'{index}/{len(matches)}')


In [8]:

output = open('data/feature_maps/gaia_matchids.pkl' , 'wb')
pickle.dump(gaia_matchids, output)
output.close()
output = open('data/feature_maps/gaia_labels.pkl' , 'wb')
pickle.dump(gaia_labels, output)
output.close()


In [9]:
print_txt("check")